In [ ]:
try:
    import sys
    sys.path.append("/Users/adameshel/Documents/Python_scripts/my_functions/")
    from helper_functions import restartkernel
    restartkernel()
except:
    print('Get the function restartkernel from helper_functions')

In [1]:
import numpy as np
import pandas as pd
import glob
from collections import Counter
import sys
from pathlib import Path

In [4]:
# path
path = '/Users/adameshel/Documents/Stuff/Basketball/Playoffs/2022/'
# xlspath = path.joinpath('NBA Playoffs.xlsx')
xlspath = Path('/Users/adameshel/Documents/Stuff/Basketball/Playoffs/2022/NBA Playoffs.xlsx')

### Type real results here

In [5]:
# real results

# results_1 = ['PHI_5', 'ATL_5', 'MIL_4', 'BKN_5', 'UTA_5', 'DEN_6', 'PHX_6', 'LAC_7']

# results_2 = ['ATL_7', 'MIL_7', 'PHX_4', 'LAC_6']
# scorers_2 = ['CHPA', 'JOEM', 'KEDU', 'DOMI']

# results_3 = ['PHX_6', 'MIL_6']
# scorers_3 = ['PAGE','KHMI']
# assisters_3 = ['CHPA','JRHO']
# rebounders_3 = ['DEAY','CLCA']


# results_4 = ['LAL_6']
# second = ['MIL']
# scorers_4 = ['GIAN']
# assisters_4 = ['JRHO']
# rebounders_4 = ['GIAN']

### Load the odds file and calculate them

In [11]:
df_xls = pd.read_excel(xlspath,header=0)
start = 1
end = 10
width = end - start

## Get the general odds
teams_l = list(df_xls.iloc[:,0].dropna().str.upper())
arr_champ = df_xls.iloc[:,1].values
min_odds = np.nanmin(arr_champ)
arr_champ = np.where(arr_champ>min_odds*20,min_odds*20,arr_champ)
arr_champ = arr_champ ** 0.5
## normalize the odds to be between 1-10
res = np.round((arr_champ - arr_champ.min())/(arr_champ.max() - arr_champ.min()) * width + start,2)
standings_d = dict(zip(teams_l, res.T))

## Get the round odds
def get_round_odds(df,Playoff_round):
    teams_l = list(df.iloc[:,Playoff_round * 2].dropna().str.upper())
    arr = df.iloc[:,Playoff_round * 2 +1].dropna().values# * team_weight_d[Playoff_round]
    round_standings_d = dict(zip(teams_l, arr.T))
    return round_standings_d

## define weights of matches in respect to rounds
team_weight_d = {
    1:3,
    2:4,
    3:5,
    4:6
}

## define the values of the score of a match
score_constant_d = {
    1:2,
    2:3,
    3:4,
    4:4
}

## define the values of the player bet
player_constant_d = {
    2:2,
    3:2,
    4:3
}

### Championship odds

In [6]:
standings_d

{'PHX': 1.0,
 'MIL': 1.69,
 'BKN': 2.19,
 'MIA': 2.64,
 'GSW': 2.64,
 'BOS': 3.04,
 'PHI': 3.59,
 'MEM': 3.59,
 'UTA': 4.83,
 'DAL': 4.83,
 'DEN': 4.83,
 'MIN': 9.69,
 'TOR': 9.69,
 'CHI': 10.0,
 'ATL': 10.0,
 'NOP': 10.0}

### Specific round odds

In [7]:
get_round_odds(df_xls,1)

{'UTA': 1.3,
 'DAL': 3.5,
 'MEM': 1.25,
 'MIN': 3.75,
 'PHI': 1.5,
 'TOR': 2.5,
 'GSW': 1.4,
 'DEN': 3.0,
 'BOS': 1.65,
 'BKN': 2.15,
 'MIL': 1.1,
 'CHI': 6.5,
 'MIA': 1.27,
 'ATL': 4.0,
 'PHX': 1.04,
 'NOP': 11.0}

### From here on DO NOT change anything

In [28]:
late_penalty = 3

def split_at(s, c, n):
    '''Function for splitting strings (first arg). The last argument is the position of the
        selected seperator (second arg) in the string.'''
    words = s.split(c)
    return c.join(words[:n]), c.join(words[n:])

def calc_scores(Playoff_round):
    TEAM = team_weight_d[Playoff_round] # weight
    SCORE = score_constant_d[Playoff_round]
    champ_score = 0
    second_score = 0
    if Playoff_round==1: 
        print('ROUND 1:\n\n')
        ROUND = 'first_round' + '/'
        results = results_1
        print('POINTS\nTeam: %s, Score: %s\n' %(TEAM,SCORE))

    elif Playoff_round==2:
        print('ROUND 2:\n\n')
        ROUND = 'second_round' + '/'
        BEST_SCORER = player_constant_d[Playoff_round]
        results = results_2
        scorers = scorers_2
        print('POINTS\nTeam: %s,\
Score: %s, Scorer:%s\n' %(TEAM,SCORE,BEST_SCORER))

    elif Playoff_round==3:
        print('ROUND 3:\n\n')
        ROUND = 'third_round' + '/'
        BEST_SCORER = player_constant_d[Playoff_round]
        BEST_ASSISTER = player_constant_d[Playoff_round]
        BEST_REBOUNDER = player_constant_d[Playoff_round]
        results = results_3
        scorers = scorers_3
        assisters = assisters_3
        rebounders = rebounders_3
        print('POINTS\nTeam: %s, Score: %s,\
Scorer: %s, Assister: %s, Rebounder: %s \n' %(TEAM,
                                              SCORE,
                                              BEST_SCORER,
                                              BEST_ASSISTER,
                                              BEST_REBOUNDER))

    elif Playoff_round==4: 
        print('FINALS:\n\n')
        ROUND = 'finals' + '/'
        BEST_SCORER = player_constant_d[Playoff_round]
        BEST_ASSISTER = player_constant_d[Playoff_round]
        BEST_REBOUNDER = player_constant_d[Playoff_round]
        CHAMP = 10 # weight
        results = results_4
        scorers = scorers_4
        assisters = assisters_4
        rebounders = rebounders_4
        print('POINTS\nTeam: %s, Score: %s,\
        Scorer: %s, Assister: %s, Rebounder: %s \n' %(TEAM,
                                                     SCORE,
                                                     BEST_SCORER,
                                                     BEST_ASSISTER,
                                                     BEST_REBOUNDER))
        
        first_files = sorted(glob.glob(path + 'first_round/' + '*.txt'))
        list_of_names = []
        list_of_scores = []
        list_of_scores_second = []
        for i, file in enumerate(first_files):
            champ_score = 0
            second_score = 0
            f = open(file,'r')
            lines = f.read().splitlines()
            last_line = lines[-1]
            if last_line != '':
                f.close()   
                f = open(file, "a+")
                f.write("\r\n")
            f.close()
            
            f = open(file)
            for j,line in enumerate(f):
                line = line.upper()
                if j==0:
                    name = line[:-1]
                    while ' ' in name[-1]:
                        name = name[:-1]
                    print(name)
                    continue
                if 'CHAMP' in line:
                    print(line)
                    champ_bet = split_at(line,',',1)[1]
                    champ_bet = champ_bet[0:3]
                    if champ_bet in results[0]:
                        champ_score = standings_d[champ_bet] * CHAMP
                        print('Ratio rewarded for early bet:')
                        print(str(CHAMP) + ' * ' +\
                              str(round(standings_d[split_at(results[0],'_',1)[0]] ,2)))
                        print('\n')
                    list_of_scores.append(champ_score)
                    list_of_names.append(name)
                if 'SECOND' in line:
                    print(line)
                    second_bet = split_at(line,',',1)[1]
                    second_bet = second_bet[0:3]
                    if second_bet==second[0]:
                        second_score = standings_d[second_bet] * (CHAMP / 2)
                        print('Ratio rewarded for early second place bet:')
                        print(str(CHAMP/2) + ' * ' +\
                              str(round(standings_d[second_bet] ,2)))
                        print('\n')
                    list_of_scores_second.append(second_score)
                    f.close()
                    break
                else:
                    continue
                
        d_champ = dict(zip(list_of_names, list_of_scores))
        d_second = dict(zip(list_of_names, list_of_scores_second))
                    
    round_standings_d = get_round_odds(df_xls,Playoff_round) 
    print('Odds for this round:')
    print(round_standings_d)
    print('\n')
    files = sorted(glob.glob(path + ROUND + '*.txt'))
    list_of_names = []
    list_of_scores = []
    for i, file in enumerate(files):
        my_team_result = 0
        my_score_result = 0
        my_best_scorer_result = 0
        my_best_assister_result = 0
        my_best_rebounder_result = 0
        my_penalty_result = 0
        f = open(file,'r')
        lines = f.read().splitlines()
        last_line = lines[-1]
        if last_line != '':
            f.close()   
            f = open(file, "a+")
            f.write("\r\n")
        f.close()
        
        f = open(file)
        for j,line in enumerate(f):
            line = line.upper()
            if j==0:
                name = line[:-1]
                while ' ' in name[-1]:
                    name = name[:-1]
                print(name)
                list_of_names.append(name)
                continue
            if '%' in line:
                continue
            if ',' in line and '_' in line:
                team_bet = split_at(split_at(line,',',1)[1],'_',1)[0]
                if team_bet not in standings_d.keys():
                    raise Exception("Team name %s was written incorrectly by %s" %(team_bet,name))
                opponent1 = split_at(split_at(line,',',1)[0],'_',1)[0]
                opponent2 = split_at(split_at(line,',',1)[0],'_',1)[1]
                score_bet = split_at(split_at(line,',',1)[1],'_',1)[1][0]
                ## to speed up move all the above three lines under the `if team_bet in result:`
                for result in results:
                    norm_base_for_match = min(round_standings_d[opponent1],
                                              round_standings_d[opponent2])
                    if team_bet in result:       
                        my_team_result += round(
                            TEAM * (round_standings_d[split_at(result,'_',1)[0]] / norm_base_for_match),
                            2
                        )
                        print('Ratio rewarded for the series ' + str(split_at(line,',',1)[0]) + ':')
                        print(str(TEAM) + ' * ' +\
                              str(round(round_standings_d[split_at(result,'_',1)[0]] / norm_base_for_match,2)))
                        # check if score is correct
                        if score_bet in result:
                            my_score_result += SCORE
            if 'SCORERS:' in line:
                players = split_at(line,':',1)[1][:-1].split(',')
                for player in players:
                    if player in scorers:
                        my_best_scorer_result += BEST_SCORER
            if 'ASSISTERS:' in line:
                players = split_at(line,':',1)[1][:-1].split(',')
                for player in players:
                    if player in assisters:
                        my_best_assister_result += BEST_ASSISTER
            if 'REBOUNDERS:' in line:
                players = split_at(line,':',1)[1][:-1].split(',')
                for player in players:
                    if player in rebounders:
                        my_best_rebounder_result += BEST_REBOUNDER 
            if 'PENALTY:' in line:
                penalty = int(split_at(line,':',1)[1][:-1].split(',')[0]) * late_penalty
                my_penalty_result += penalty

            if Playoff_round==4: 
                champ_score = d_champ[name]
                second_score = d_second[name]
        temp_total = round(my_team_result +\
                     my_score_result +\
                     my_best_scorer_result +\
                     my_best_assister_result +\
                     my_best_rebounder_result +\
                     champ_score +\
                     second_score -\
                     my_penalty_result,
                           2
                    )
        list_of_scores.append(temp_total)

        f.close()

        if Playoff_round==1: 
            if my_penalty_result==0:
                print('Team: ' + str(my_team_result),
                      'Score: ' + str(my_score_result), 
                      'Total for this round: ' + str(temp_total),
                      '\n')
            else:
                print('Team: ' + str(my_team_result),
                      'Score: ' + str(my_score_result),
                      'Penalty: ' + str(my_penalty_result),
                      'Total for this round: ' + str(temp_total),
                      '\n')

        elif Playoff_round==2:
            if my_penalty_result==0:
                print('Team: ' + str(my_team_result),
                  'Score: ' + str(my_score_result),
                  'Scorer: ' + str(my_best_scorer_result),
                  'Total for this round: ' + str(temp_total),
                  '\n')
            else:
                print('Team: ' + str(my_team_result),
                  'Score: ' + str(my_score_result),
                  'Scorer: ' + str(my_best_scorer_result),
                  'Penalty: ' + str(my_penalty_result),
                  'Total for this round: ' + str(temp_total),
                  '\n')
        elif Playoff_round==3:
            if my_penalty_result==0:
                print('Team: ' + str(my_team_result),
                      'Score: ' + str(my_score_result),
                      'Scorer: ' + str(my_best_scorer_result),
                      'Assister: ' + str(my_best_assister_result),
                      'Rebounder: ' + str(my_best_rebounder_result),
                      'Total for this round: ' + str(temp_total),
                      '\n')
            else:
                print('Team: ' + str(my_team_result),
                      'Score: ' + str(my_score_result),
                      'Scorer: ' + str(my_best_scorer_result),
                      'Assister: ' + str(my_best_assister_result),
                      'Rebounder: ' + str(my_best_rebounder_result),
                      'Penalty: ' + str(my_penalty_result),
                      'Total for this round: ' + str(temp_total),
                      '\n')
        else:
            if my_penalty_result==0:
                print('Team: ' + str(my_team_result),
                      'Score: ' + str(my_score_result),
                      'Scorer: ' + str(my_best_scorer_result),
                      'Assister: ' + str(my_best_assister_result),
                      'Rebounder: ' + str(my_best_rebounder_result),
                      'Champion: ' + str(champ_score),
                      'Second: ' + str(second_score),
                      'Total for this round: ' + str(temp_total),
                      '\n')
            else:
                print('Team: ' + str(my_team_result),
                  'Score: ' + str(my_score_result),
                  'Scorer: ' + str(my_best_scorer_result),
                  'Assister: ' + str(my_best_assister_result),
                  'Rebounder: ' + str(my_best_rebounder_result),
                  'Champion: ' + str(champ_score),
                  'Second: ' + str(second_score),
                  'Penalty: ' + str(my_penalty_result),
                  'Total for this round: ' + str(temp_total),
                  '\n')
            
    print('\n\n\n')
    d = dict(zip(list_of_names, list_of_scores))
    return d

def calc_total(round_number):
    if round_number == 1:
        d_comb = calc_scores(Playoff_round=round_number)
    elif round_number == 2:
        d_1 = calc_scores(Playoff_round=round_number-1)
        d_comb = dict(Counter(d_1)+Counter(calc_scores(Playoff_round=round_number)))
    elif round_number == 3:
        d_1 = calc_scores(Playoff_round=round_number-2)
        d_2 = dict(Counter(d_1)+Counter(calc_scores(Playoff_round=round_number-1)))
        d_comb = dict(Counter(d_2)+Counter(calc_scores(Playoff_round=round_number)))
    elif round_number == 4:
        d_1 = calc_scores(Playoff_round=round_number-3)
        d_2 = dict(Counter(d_1)+Counter(calc_scores(Playoff_round=round_number-2)))
        d_3 = dict(Counter(d_2)+Counter(calc_scores(Playoff_round=round_number-1)))
        d_comb = dict(Counter(d_3)+Counter(calc_scores(Playoff_round=round_number)))
    else:
        print('There is a problem with the scores dictionaries')
    print('TOTALS:\n\n')
    sort_orders = sorted(d_comb.items(), key=lambda x: x[1], reverse=True)
    pos_count = 0
    for i in sort_orders:
        pos_count+=1
        print(str(pos_count) + '. '+ str(i[0]))
        print('    ' + str(i[1]))
        print('\n')
#     for key in d_comb.keys():
#         print(key, '\t')
#         print(d_comb[key], '\n')

In [29]:
calc_total(round_number=4)

ROUND 1:


POINTS
Team: 3, Score: 2

Odds for this round:
{'UTA': 1.3, 'DAL': 3.5, 'MEM': 1.25, 'MIN': 3.75, 'PHI': 1.5, 'TOR': 2.5, 'GSW': 1.4, 'DEN': 3.0, 'BOS': 1.65, 'BKN': 2.15, 'MIL': 1.1, 'CHI': 6.5, 'LAC': 1.0, 'POR': 1.0, 'PHX': 1.0, 'LAL': 1.0, 'NYK': 1.0, 'WAS': 1.0, 'ATL': 1.0, 'MIA': 1.0}


ADAM_ESHEL
Ratio rewarded for the series UTA_MEM:
3 * 1.04
Ratio rewarded for the series LAC_DAL:
3 * 1.0
Ratio rewarded for the series DEN_POR:
3 * 3.0
Ratio rewarded for the series PHI_WAS:
3 * 1.5
Ratio rewarded for the series NYK_ATL:
3 * 1.0
Ratio rewarded for the series BKN_BOS:
3 * 1.3
Team: 26.53 Score: 6 Total for this round: 32.53 

ALON_KESTECHER
Ratio rewarded for the series UTA_MEM:
3 * 1.04
Ratio rewarded for the series LAC_DAL:
3 * 1.0
Ratio rewarded for the series DEN_POR:
3 * 3.0
Ratio rewarded for the series PHI_WAS:
3 * 1.5


Exception: Team name NYK was not written properly by ALON_KESTECHER

# Print all bets for a specific round

In [8]:
first_files = sorted(glob.glob(path + 'first_round/' + '*.txt'))
list_of_names = []
list_of_scores = []
for i, file in enumerate(first_files):
    print('\n')
    champ_score = 0
    f = open(file,'r')
    lines = f.read().splitlines()
    last_line = lines[-1]
    if last_line != '':
        f.close()   
        f = open(file, "a+")
        f.write("\r\n")
    f.close()
    f = open(file)
    for j,line in enumerate(f):
        line = line.upper()
        print(line)



AVIV



CHAMP,MIL

SECOND,GSW



MIA_ATL,MIA_6

BOS_BKN,BKN_7

MIL_CHI,MIL_6

PHI_TOR,PHI_6



PHX_NOP,PHX_5

MEM_MIN,MEM_6

GSW_DEN,GSW_7

DAL_UTA,UTA_5



DAN



CHAMP,MIL

SECOND,PHX



MIA_XXX,MIA_4

BOS_BKN,BOS_7

MIL_CHI,MIL_5

PHI_TOR,PHI_6



PHX_XXX,PHX_4

MEM_MIN,MEM_5

GSW_DEN,GSW_7

DAL_UTA,DAL_7



SHAI



CHAMP,PHX

SECOND,BOS



MIA_ATL,ATL_6

BOS_BKN,BOS_5

MIL_CHI,MIL_4

PHI_TOR,TOR_6



PHX_NOP,PHX_4

MEM_MIN,MEM_5

GSW_DEN,DEN_6

DAL_UTA,DAL_7



OFER_KLONER



CHAMP,BKN

SECOND,DEN



MIA_ATL,MIA_7

BOS_BKN,BKN_6

MIL_CHI,MIL_5

PHI_TOR,PHI_4



PHX_NOP,PHX_4

MEM_MIN,MEM_5

GSW_DEN,DEN_7

DAL_UTA,UTA_4



GILAD_SOKOLOVER



CHAMP,MIL

SECOND,PHX







PHI_TOR,PHI_6





MEM_MIN,MIN_6

GSW_DEN,GSW_6

DAL_UTA,UTA_6



ADAM



CHAMP,PHX

SECOND,BOS



MIA_ATL,MIA_5

BOS_BKN,BOS_7

MIL_CHI,MIL_4

PHI_TOR,PHI_5



PHX_NOP,PHX_4

MEM_MIN,MEM_6

GSW_DEN,GSW_7

DAL_UTA,UTA_6



SHAHAF



CHAMP, PHX

SECOND, MIL



MIA_ATL,MIA_5

BOS_BKN,BOS_7

MIL_CHI,MIL_5

PHI_TOR,PHI